In [1]:
import pandas as pd
from collections import Counter
import re

In [4]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/Solr-dsi-final_v1.csv')

In [2]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/raw/solr-dsi-domain_only-04-05-2017-cleaned.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292147 entries, 0 to 292146
Data columns (total 12 columns):
_version_    292147 non-null int64
cache        3956 non-null object
segment      292147 non-null int64
digest       292147 non-null object
tstamp       292147 non-null object
url          292147 non-null object
anchor       31946 non-null object
content      292147 non-null object
id           292147 non-null object
title        292147 non-null object
boost        292147 non-null float64
domain       292147 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 26.7+ MB


In [7]:
df = df.dropna(subset =['content'])

In [356]:
msk = np.random.rand(len(df)) < 0.2
train = df[msk]
test = df[~msk]

In [357]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69856 entries, 0 to 89752
Data columns (total 12 columns):
Unnamed: 0    69856 non-null int64
_version_     69856 non-null int64
cache         755 non-null object
segment       69856 non-null int64
digest        69856 non-null object
tstamp        69856 non-null object
url           69856 non-null object
anchor        16322 non-null object
content       69856 non-null object
id            69856 non-null object
title         69081 non-null object
boost         69856 non-null float64
dtypes: float64(1), int64(3), object(8)
memory usage: 6.9+ MB


In [358]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17602 entries, 5 to 89749
Data columns (total 12 columns):
Unnamed: 0    17602 non-null int64
_version_     17602 non-null int64
cache         197 non-null object
segment       17602 non-null int64
digest        17602 non-null object
tstamp        17602 non-null object
url           17602 non-null object
anchor        4174 non-null object
content       17602 non-null object
id            17602 non-null object
title         17368 non-null object
boost         17602 non-null float64
dtypes: float64(1), int64(3), object(8)
memory usage: 1.7+ MB


In [347]:
articles = df['content']

In [359]:
articles = test['content']

In [360]:
articles_split = [re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', item) for item in articles]

In [361]:
sentences = [item for sublist in articles_split for item in sublist]

In [362]:
len(sentences)

481592

import re
text = "Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of .9 it isn't."
t = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

for r in t:
    print r

In [363]:
count = Counter(sentences)

top_200 = count.most_common(200)
dsi_words, cnt = zip(*top_200)
tot = sum(cnt)
dsi_fq = [float(i)*1.0/tot for i in cnt]
dsi_dict_org = dict(zip(dsi_words, dsi_fq))

In [364]:
top_200 = count.most_common(200)
dsi_words, cnt = zip(*top_200)
tot = sum(cnt)
dsi_fq = [float(i)*1.0/tot for i in cnt]
dsi_dict_org1 = dict(zip(dsi_words, dsi_fq))

In [301]:
top_200 = count.most_common(200)
dsi_words, cnt = zip(*top_200)
tot = sum(cnt)
dsi_fq = [float(i)*1.0/tot for i in cnt]
dsi_dict_org2 = dict(zip(dsi_words, dsi_fq))

In [365]:
get_cosine(dsi_dict_org, dsi_dict_org1)

0.9903353619035669

In [302]:
get_cosine(dsi_dict_org, dsi_dict_org2)

1.0

In [233]:
sort = count.most_common(len(count))

In [234]:
for t in sort[:200]:
    print t[0], len(t[0]), t[1]

. 1 48421
Please check your browser settings or contact your system administrator. 72 4946
All rights reserved. 20 4117
All Rights Reserved. 20 2196
New to Twitter? 15 1821
Remember me · Forgot password? 31 1772
Log in Have an account? 23 1772
9 Must-Have Skills You Need to Become a Data Scientist - Nov 22, 2014. 70 1645
R vs Python for Data Science: The Winner is ... 47 1640
- May 26, 2015 . 16 1623
Top 10 Machine Learning Projects on Github - Dec 14, 2015. 58 1343
More Data Science Humor and Cartoons - Dec 23, 2015. 52 1341
2. 2 1320
Data Exploration in Preparation for Modeling A Concise Overview of Recent Advances in Chat... 93 1287
AbsolutData: Data Analyst (must be proficient... 48 1287
About KDnuggets   Subscribe to KDnuggets News X 48 1285
The Star Wars social networks - who is the central character? 61 1205
- Dec 25, 2015 . 16 1205
The Art of Data Science: The Skills You Need and How to Get Them - Dec 28, 2015 . 81 1204
3. 2 1113
Public Knowledge Graph – small guys unite CEOs P

In [71]:
cnt = 0
cnt_list = []
prev_cnt = 0
for t in sort:
    if prev_cnt == 0:
        prev_cnt = t[1]
        cnt = 1
    elif prev_cnt != t[1]:
        cnt_list.append((prev_cnt, cnt))
        prev_cnt = t[1]
        cnt = 1
    else:        
        cnt += 1
cnt_list.append((prev_cnt, cnt))

print len(cnt_list)
print cnt_list

In [235]:
sen, cnt = zip(*sort)

In [236]:
sentences_counted = [item for item in sen if len(item.split()) > 1]

In [237]:
words = [item.split() for item in sentences_counted]

In [238]:
from sklearn.feature_extraction import stop_words

In [239]:
import re
def dsi_title_letters_removal(x):
    s = re.sub('[^a-zA-Z]+', '', x)
    if(len(s) > 0):
        s = s.strip().lower()
        if s not in stop_words.ENGLISH_STOP_WORDS:
            return s
        else:
            return None
    return None

In [240]:
flatten = [dsi_title_letters_removal(item) for sublist in words for item in sublist if dsi_title_letters_removal(item) != None ]

In [241]:
print len(flatten)

25989433


In [242]:
count = Counter(flatten)

In [243]:
top_200 = count.most_common(200)

In [194]:
dsi_stop_words = ['the', 'new', 'i', 'in', 'contact', 'use', 'home', 'about', 'us', 'a', 'it', 'this', 'blog'
                 'more', 'l', 'all', 'for', 'using', 'like', 'sign', 'view', 'web', 'best', 'how', 'support',
                 'help', 'marketing', 'you', 'we', 'need', 'time', 'university', 'social', 'content', 'online',
                 'media', 'post', 'service', 'program', 'work', 'video', 'people', 'email', 'comments', 'resources',
                 'read', 'company', 'google', 'open', 'customer', 'mobile']

In [244]:
dsi_words, cnt = zip(*top_200)
tot = sum(cnt)
dsi_fq = [float(i)*1.0/tot for i in cnt]
dsi_dict2 = dict(zip(dsi_words, dsi_fq))

In [222]:
dsi_words, cnt = zip(*top_200)
tot = sum(cnt)
dsi_fq = [float(i)*1.0/tot for i in cnt]
dsi_dict1 = dict(zip(dsi_words, dsi_fq))

In [206]:
dsi_words, cnt = zip(*top_200)
tot = sum(cnt)
dsi_fq = [float(i)*1.0/tot for i in cnt]
dsi_dict = dict(zip(dsi_words, dsi_fq))

In [226]:
import math
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    #print intersection
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [227]:
get_cosine(dsi_dict, dsi_dict1)

0.9966335188115738

In [245]:
get_cosine(dsi_dict, dsi_dict2)

0.9999926088359031